Data cleaning for data related to enrollments of students in Canada
Data taken from Statscan - Table: 37-10-0011-01 (formerly CANSIM 477-0029)

In [1]:
import pandas as pd

In [2]:
# read csv file
df = pd.read_csv("37100018.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_17348\3798948813.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("37100018.csv")


In [3]:
# Assuming you have a DataFrame named df
memory_in_bytes = df.memory_usage(deep=True).sum()
memory_in_mb = memory_in_bytes / (1024 ** 2)  # Convert bytes to MB

print(f"The DataFrame takes up {memory_in_mb:.2f} MB in memory.")

The DataFrame takes up 4905.61 MB in memory.


In [4]:
# 4.6 mln records
df.shape

(4645275, 21)

In [6]:
# clean data by deleting not usefull data
df_clean = df.drop(columns = ['DGUID', 'UOM', 'UOM_ID', 'TERMINATED', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS'])

df_clean = df_clean[df_clean['Selected statistics'] == 'Number']
df_clean = df_clean.drop(columns = ['Selected statistics'])

In [7]:
# rename some data
df_clean = df_clean.replace(to_replace=r'^Total.*', value='Total', regex=True)

# rename columns
df_clean = df_clean.rename(columns = {'REF_DATE': 'Date', 'GEO': 'Location', 'Status of student in Canada': 'Status of student', 'VALUE': 'Value', 'International Standard Classification of Education (ISCED)': 'ISCED'})

# reconfigure date column
df_clean['Date'] = df_clean['Date'].str.split('/').str[0].astype(int)

In [12]:
# showcase how data looks like
df_clean

,Date,Location,Institution type,Registration status,ISCED,Field of study,Gender,Status of student,Value
0,1992,Canada,Total,Total,Total,Total,Total,Total,1355451.0
2,1992,Canada,Total,Total,Total,Total,Total,Canadian students,1190811.0
4,1992,Canada,Total,Total,Total,Total,Total,International students,40620.0
6,1992,Canada,Total,Total,Total,Total,Total,"Not reported, status of student in Canada",124023.0
8,1992,Canada,Total,Total,Total,Total,Man,Total,608616.0
...,...,...,...,...,...,...,...,...,...
4645265,2021,Territories,College,Part-time student,"Not applicable, International Standard Classif...",Personal improvement and leisure [0],Woman,Canadian students,6.0
4645267,2021,Territories,College,Part-time student,"Not applicable, International Standard Classif...",Unclassified,Total,Total,6.0
4645269,2021,Territories,College,Part-time student,"Not applicable, International Standard Classif...",Unclassified,Total,Canadian students,6.0
4645271,2021,Territories,College,Part-time student,"Not applicable, International Standard Classif...",Unclassified,Woman,Total,6.0


In [18]:
df_clean.columns

Index(['Date', 'Location', 'Institution type', 'Registration status', 'ISCED',
       'Field of study', 'Gender', 'Status of student', 'Value'],
      dtype='object')

In [22]:
# Delete row with "Canada" or "Total" in them because they duplicate calculations
df_filtered = df_clean[~df_clean.astype(str).apply(lambda x: x.str.contains('Total|Canada', case=False, na=False)).any(axis=1).values]

In [23]:
df_filtered

,Date,Location,Institution type,Registration status,ISCED,Field of study,Gender,Status of student,Value
25736,1992,Newfoundland and Labrador,University,Full-time student,Bachelor's or equivalent,Personal improvement and leisure [0],Woman,Canadian students,NaN
25745,1992,Newfoundland and Labrador,University,Full-time student,Bachelor's or equivalent,Education [1],Man,Canadian students,153.0
25747,1992,Newfoundland and Labrador,University,Full-time student,Bachelor's or equivalent,Education [1],Man,International students,NaN
25750,1992,Newfoundland and Labrador,University,Full-time student,Bachelor's or equivalent,Education [1],Woman,Canadian students,543.0
25752,1992,Newfoundland and Labrador,University,Full-time student,Bachelor's or equivalent,Education [1],Woman,International students,NaN
...,...,...,...,...,...,...,...,...,...
4645239,2021,Territories,College,Part-time student,Bachelor's or equivalent,Other [12],Woman,Canadian students,6.0
4645242,2021,Territories,College,Part-time student,Bachelor's or equivalent,Other [12],Gender unknown,Canadian students,3.0
4645261,2021,Territories,College,Part-time student,"Not applicable, International Standard Classif...",Personal improvement and leisure [0],Man,Canadian students,9.0
4645265,2021,Territories,College,Part-time student,"Not applicable, International Standard Classif...",Personal improvement and leisure [0],Woman,Canadian students,6.0


In [24]:
# save clean data
df_filtered.to_csv('enrollments_data.csv', index = False)